In [926]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [927]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime



In [1655]:
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, AGE, RANK AS COMPETITION_RANK, EVENT, DOB, COUNTRY, CATEGORY_EVENT, GENDER, COMPETITION, DATE
FROM `saa-analytics.results.saa_full` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='FOUL' AND RANK!='DNS' AND RESULT!='DNS' AND RESULT!='DNF'  AND RESULT!='DNQ' AND RESULT IS NOT NULL
"""

athletes = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1656]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,LIM JUN YI MATTHEW,DQ,None,13.0,110m Hurdles (0.991m),None,None,Hurdles,Male,NSG,2023
1,LUI LUCAS,DQ,None,15.0,400m Hurdles (0.914m),None,None,Hurdles,Male,NSG,2023
2,LIEU YEA JAY ERWIN,DQ,None,10.0,5000m Race Walk,None,None,Walk,Male,NSG,2023
3,ZHENG YUJIE BRANDON,DQ,None,27.0,3000m Race Walk,None,None,Walk,Male,NSG,2023
4,MARCUS ONG PINRONG,DQ,None,15.0,3000m Race Walk,None,None,Walk,Male,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...
21590,"{435: 'Peh, Ying Xuan Ulyss', 324: 'Tan, Codee...",1:02.81,None,12,Girls 10-12 4x100 Meter Relay U13,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21591,"{799: 'Yu Xuan, Lua', 802: 'Annabelle Jiamin, ...",58.60,None,4,Girls 10-12 4x100 Meter Relay U13,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21592,"{983: 'Shah, Sharleez Eimann', 985: 'Pugsley, ...",1:09.86,None,18,Girls 10-12 4x100 Meter Relay U13,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21593,"{1309: 'Ong Chun Yi, Avelyn', 1308: 'Verena Ki...",1:03.22,None,14,Girls 10-12 4x100 Meter Relay U13,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023


In [1657]:
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m run')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110 Meter Hurdles.+', value='110m hurdles')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Hurdles.+', value='100m hurdles')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Hurdles.+', value='400m hurdles')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500m Race Walk.+', value='1500m race walk')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Shot Put.+', value='Shot put')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Discus.+', value='Discus throw')

mask = athletes['EVENT'].str.contains(r'Discus', na=True)
athletes.loc[mask, 'EVENT'] = 'Discus throw'

mask = athletes['EVENT'].str.contains(r'Shot', na=True)
athletes.loc[mask, 'EVENT'] = 'Shot put'

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'EVENT'] = 'Javelin throw'



athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw.+', value='Javelin throw')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump.+', value='Long jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')


athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump.+', value='High jump')

athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase.+', value='3000m steeplechase')

mask = athletes['EVENT'].str.contains(r'High', na=True)
athletes.loc[mask, 'EVENT'] = 'High jump'


# correct javelin category
mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'




In [1658]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,LIM JUN YI MATTHEW,DQ,None,13.0,110m Hurdles (0.991m),None,None,Hurdles,Male,NSG,2023
1,LUI LUCAS,DQ,None,15.0,400m Hurdles (0.914m),None,None,Hurdles,Male,NSG,2023
2,LIEU YEA JAY ERWIN,DQ,None,10.0,5000m Race Walk,None,None,Walk,Male,NSG,2023
3,ZHENG YUJIE BRANDON,DQ,None,27.0,3000m Race Walk,None,None,Walk,Male,NSG,2023
4,MARCUS ONG PINRONG,DQ,None,15.0,3000m Race Walk,None,None,Walk,Male,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...
21590,"{435: 'Peh, Ying Xuan Ulyss', 324: 'Tan, Codee...",1:02.81,None,12,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21591,"{799: 'Yu Xuan, Lua', 802: 'Annabelle Jiamin, ...",58.60,None,4,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21592,"{983: 'Shah, Sharleez Eimann', 985: 'Pugsley, ...",1:09.86,None,18,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023
21593,"{1309: 'Ong Chun Yi, Avelyn', 1308: 'Verena Ki...",1:03.22,None,14,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023


In [1659]:
athletes.to_csv('athletes.csv', sep=',', encoding='utf-8-sig', index=False)


In [1660]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND RANK='3'
"""

benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1661]:
benchmarks

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final
1,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final
2,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final
3,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final
4,Robi Syianturi,14:43.45,3,5000m,Long,Male,SEA Games,Final
5,Tan Htike Soe,31:25.55,3,10000m,Sprint,Male,SEA Games,Final
6,John Cabang,13.855,3,110m hurdles,Hurdles,Male,SEA Games,Final
7,Calvin Quek,50.75,3,400m hurdles,Hurdles,Male,SEA Games,Final
8,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final
9,"Jonathan Nyepa, Khairul Hafiz Jantan, Mohama...",39.36,3,4 x 100m relay,Relay,Male,SEA Games,Final


In [1662]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [1663]:
benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [1664]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,10.443,100m,Male
1,21.02,200m,Male
2,46.63,400m,Male
3,1:53.86,800m,Male
4,14:43.45,5000m,Male
5,31:25.55,10000m,Male
6,13.855,110m hurdles,Male
7,50.75,400m hurdles,Male
8,8:55.05,3000m steeplechase,Male
9,39.36,4 x 100m relay,Male


In [1665]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()    
    
    if any(s in string for s in l)==True:
        
        if 'm' in metric:
            
            metric=metric.replace('m', '')
            output=float(str(metric))
            
        elif 'GR' in metric:
            
            metric=metric.replace('GR', '')
            output=float(str(metric))


        else:
    
            output=float(str(metric))
        
    else:
        
        searchstring = ":"
        searchstring2 = "."
        substring=str(metric)
        count = substring.count(searchstring)
        count2 = substring.count(searchstring2)
            
        if count==0:
            output=float(substring)
                        
             
        elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                                                
            time=str(metric)
            h, m ,s = time.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
        elif (count==1 and count2==1):
            
            m,s = metric.split(':')
            output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
            
            if output==229.90:
                print(metric, m, s, output, 'here')

                     
        elif (count==1 and count2==2):
            
            metric = metric.replace(".", ":", 1)
            
            h,m,s = metric.split(':')            
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
        elif (count==2 and count2==0):
            
            h,m,s = metric.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                
    return output

In [1666]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    
    metric=benchmarks.iloc[rowIndex,0]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

In [1667]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,10.443,100m,Male,10.443
1,21.02,200m,Male,21.020
2,46.63,400m,Male,46.630
3,1:53.86,800m,Male,113.860
4,14:43.45,5000m,Male,883.450
5,31:25.55,10000m,Male,1885.550
6,13.855,110m hurdles,Male,13.855
7,50.75,400m hurdles,Male,50.750
8,8:55.05,3000m steeplechase,Male,535.050
9,39.36,4 x 100m relay,Male,39.360


In [1668]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put', na=True)

benchmarks.loc[mask, '2pc']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '35pc']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5pc']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2pc']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '35pc']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5pc']=benchmarks['Metric']*1.05


benchmarks.iloc[5, [1]]='10000m run'
benchmarks.iloc[28, [1]]='10000m run'
benchmarks.iloc[26, [1]]='1500m'


In [1669]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc
0,10.443,100m,Male,10.443,10.65186,10.808505,10.96515
1,21.02,200m,Male,21.020,21.44040,21.755700,22.07100
2,46.63,400m,Male,46.630,47.56260,48.262050,48.96150
3,1:53.86,800m,Male,113.860,116.13720,117.845100,119.55300
4,14:43.45,5000m,Male,883.450,901.11900,914.370750,927.62250
5,31:25.55,10000m run,Male,1885.550,1923.26100,1951.544250,1979.82750
6,13.855,110m hurdles,Male,13.855,14.13210,14.339925,14.54775
7,50.75,400m hurdles,Male,50.750,51.76500,52.526250,53.28750
8,8:55.05,3000m steeplechase,Male,535.050,545.75100,553.776750,561.80250
9,39.36,4 x 100m relay,Male,39.360,40.14720,40.737600,41.32800


In [1670]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc
0,10.443,100m,Male,10.443,10.65186,10.808505,10.96515
1,21.02,200m,Male,21.020,21.44040,21.755700,22.07100
2,46.63,400m,Male,46.630,47.56260,48.262050,48.96150
3,1:53.86,800m,Male,113.860,116.13720,117.845100,119.55300
4,14:43.45,5000m,Male,883.450,901.11900,914.370750,927.62250
5,31:25.55,10000m run,Male,1885.550,1923.26100,1951.544250,1979.82750
6,13.855,110m hurdles,Male,13.855,14.13210,14.339925,14.54775
7,50.75,400m hurdles,Male,50.750,51.76500,52.526250,53.28750
8,8:55.05,3000m steeplechase,Male,535.050,545.75100,553.776750,561.80250
9,39.36,4 x 100m relay,Male,39.360,40.14720,40.737600,41.32800


In [1671]:
df = athletes.reset_index().merge(benchmarks.reset_index(), on=['EVENT','GENDER'], how='left')


In [1672]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc
0,0,LIM JUN YI MATTHEW,DQ,None,13.0,110m Hurdles (0.991m),None,None,Hurdles,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN
1,1,LUI LUCAS,DQ,None,15.0,400m Hurdles (0.914m),None,None,Hurdles,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN
2,2,LIEU YEA JAY ERWIN,DQ,None,10.0,5000m Race Walk,None,None,Walk,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ZHENG YUJIE BRANDON,DQ,None,27.0,3000m Race Walk,None,None,Walk,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN
4,4,MARCUS ONG PINRONG,DQ,None,15.0,3000m Race Walk,None,None,Walk,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21590,21590,"{435: 'Peh, Ying Xuan Ulyss', 324: 'Tan, Codee...",1:02.81,None,12,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.1403,46.809
21591,21591,"{799: 'Yu Xuan, Lua', 802: 'Annabelle Jiamin, ...",58.60,None,4,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.1403,46.809
21592,21592,"{983: 'Shah, Sharleez Eimann', 985: 'Pugsley, ...",1:09.86,None,18,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.1403,46.809
21593,21593,"{1309: 'Ong Chun Yi, Avelyn', 1308: 'Verena Ki...",1:03.22,None,14,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.1403,46.809


In [1673]:
df[df['EVENT']=='Shot put']

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc
115,115,WONG CHOONG YIN,13.7,None,5.0,Shot put,None,None,Throw,Male,NSG,2023,17.0,17.3,17.30,16.9540,16.6945,16.435
116,116,NEO SHYH KAI,14.62,None,4.0,Shot put,None,None,Throw,Male,NSG,2023,17.0,17.3,17.30,16.9540,16.6945,16.435
117,117,LOW CHUAN YI,16.12,None,1.0,Shot put,None,None,Throw,Male,NSG,2023,17.0,17.3,17.30,16.9540,16.6945,16.435
118,118,KAN FU YI,14.71,None,2.0,Shot put,None,None,Throw,Male,NSG,2023,17.0,17.3,17.30,16.9540,16.6945,16.435
119,119,MARK TAN RUI MING,11.4,None,7.0,Shot put,None,None,Throw,Male,NSG,2023,17.0,17.3,17.30,16.9540,16.6945,16.435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20561,20561,"Go, Sharon",6.66m,24,2,Shot put,1999-11-16 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,40.0,14.44,14.44,14.1512,13.9346,13.718
20562,20562,"Yun Ning, Tan",9.44m,18,1,Shot put,2005-01-30 00:00:00.000,None,Throw,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,40.0,14.44,14.44,14.1512,13.9346,13.718
20602,20602,"Tan, Ting Jun",11.90m,26,1,Shot put,1997-04-20 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,17.0,17.3,17.30,16.9540,16.6945,16.435
20603,20603,"Pongsree, Tanawin",10.28m,17,2,Shot put,2006-02-21 00:00:00.000,None,Throw,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,17.0,17.3,17.30,16.9540,16.6945,16.435


In [1674]:
df['RESULT'] = athletes['RESULT'].replace(regex=r'–', value=np.NaN)


In [1675]:
for i in range(len(df)):
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,5]
    
    
    metric=df.iloc[rowIndex,2]
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric=='NH':
        continue
    
        
    out = convert_time(i, input_string, metric)
         
    df.loc[rowIndex, 'RESULT_CONV'] = out

In [1676]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,LIM JUN YI MATTHEW,DQ,None,13.0,110m Hurdles (0.991m),None,None,Hurdles,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,LUI LUCAS,DQ,None,15.0,400m Hurdles (0.914m),None,None,Hurdles,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,LIEU YEA JAY ERWIN,DQ,None,10.0,5000m Race Walk,None,None,Walk,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ZHENG YUJIE BRANDON,DQ,None,27.0,3000m Race Walk,None,None,Walk,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,MARCUS ONG PINRONG,DQ,None,15.0,3000m Race Walk,None,None,Walk,Male,NSG,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21590,21590,"{435: 'Peh, Ying Xuan Ulyss', 324: 'Tan, Codee...",1:02.81,None,12,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.1403,46.809,62.81
21591,21591,"{799: 'Yu Xuan, Lua', 802: 'Annabelle Jiamin, ...",58.60,None,4,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.1403,46.809,58.60
21592,21592,"{983: 'Shah, Sharleez Eimann', 985: 'Pugsley, ...",1:09.86,None,18,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.1403,46.809,69.86
21593,21593,"{1309: 'Ong Chun Yi, Avelyn', 1308: 'Verena Ki...",1:03.22,None,14,4 x 100m relay,None,None,Relay,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,32.0,44.58,44.58,45.4716,46.1403,46.809,63.22


In [1677]:
df["AGE"].fillna(0, inplace=True)
df['AGE'] = df['AGE'].astype('float')

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/2753303150.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["AGE"].fillna(0, inplace=True)


In [1678]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [1679]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [1680]:
rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Mid')|(df['CATEGORY_EVENT']=='Sprint')|(df['CATEGORY_EVENT']=='Long')|(df['CATEGORY_EVENT']=='Hurdles')|(df['CATEGORY_EVENT']=='Walk')|(df['CATEGORY_EVENT']=='Relay')|(df['CATEGORY_EVENT']=='Marathon')|(df['CATEGORY_EVENT']=='Steeple')|(df['CATEGORY_EVENT']=='Pentathlon')|(df['CATEGORY_EVENT']=='Heptathlon')|(df['CATEGORY_EVENT']=='Triathlon'))&(df['RESULT_CONV'] <= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))|(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [1681]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,DATE,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV
288,288,"LOH DING RONG, ANSON",50.25,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2023,18.0,50.02,50.02,49.0196,48.26930,47.5190,50.25
291,291,TAN JUN JIE,48.38,0.0,2.0,Discus throw,None,None,Throw,Male,NSG,2023,18.0,50.02,50.02,49.0196,48.26930,47.5190,48.38
392,392,LEE YU FOONG (LI YUFENG),51.28,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2023,18.0,50.02,50.02,49.0196,48.26930,47.5190,51.28
725,725,Tan Jun Jie,53.4,0.0,1.0,Discus throw,None,None,Throw,Male,NSG,2024,18.0,50.02,50.02,49.0196,48.26930,47.5190,53.40
726,726,Wong Choong Yin,47.59,0.0,2.0,Discus throw,None,None,Throw,Male,NSG,2024,18.0,50.02,50.02,49.0196,48.26930,47.5190,47.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18661,18661,"Yap, Sean Yee",1.71m,28.0,1,High jump,1995-05-01,None,Jump,Female,10th Club ZOOM Kindred Spirit Series 2023,2023,36.0,1.73,1.73,1.6954,1.66945,1.6435,1.71
18984,18984,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,21.02,21.02,21.4404,21.75570,22.0710,22.03
18985,18985,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,21.02,21.02,21.4404,21.75570,22.0710,22.03
18986,18986,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,10th Club ZOOM Kindred Spirit Series 2023,2023,1.0,21.02,21.02,21.4404,21.75570,22.0710,22.03


In [1682]:
mask = rslt_df['CATEGORY_EVENT'].str.contains(r'Jump|Throw', na=True)
rslt_df.loc[mask, 'Delta2'] = rslt_df['RESULT_CONV']-rslt_df['2pc']
rslt_df.loc[mask, 'Delta35'] = rslt_df['RESULT_CONV']-rslt_df['35pc']
rslt_df.loc[mask, 'Delta5'] = rslt_df['RESULT_CONV']-rslt_df['5pc']

rslt_df.loc[~mask, 'Delta2'] =  rslt_df['2pc'] - rslt_df['RESULT_CONV']
rslt_df.loc[~mask, 'Delta35'] = rslt_df['35pc'] - rslt_df['RESULT_CONV']
rslt_df.loc[~mask, 'Delta5'] = rslt_df['5pc'] - rslt_df['RESULT_CONV']



#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
rslt_df=rslt_df.loc[rslt_df['COMPETITION']!='SEA Games']

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/164990800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df.loc[mask, 'Delta2'] = rslt_df['RESULT_CONV']-rslt_df['2pc']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/164990800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df.loc[mask, 'Delta35'] = rslt_df['RESULT_CONV']-rslt_df['35pc']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/164990800.py:4: SettingWithCopyWarning: 
A value is try

In [1683]:
rslt_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,index_y,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5
288,288,"LOH DING RONG, ANSON",50.25,0.0,1.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.02,49.0196,48.26930,47.5190,50.25,1.2304,1.98070,2.7310
291,291,TAN JUN JIE,48.38,0.0,2.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.02,49.0196,48.26930,47.5190,48.38,-0.6396,0.11070,0.8610
392,392,LEE YU FOONG (LI YUFENG),51.28,0.0,1.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.02,49.0196,48.26930,47.5190,51.28,2.2604,3.01070,3.7610
725,725,Tan Jun Jie,53.4,0.0,1.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.02,49.0196,48.26930,47.5190,53.40,4.3804,5.13070,5.8810
726,726,Wong Choong Yin,47.59,0.0,2.0,Discus throw,None,None,Throw,Male,...,18.0,50.02,50.02,49.0196,48.26930,47.5190,47.59,-1.4296,-0.67930,0.0710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18661,18661,"Yap, Sean Yee",1.71m,28.0,1,High jump,1995-05-01,None,Jump,Female,...,36.0,1.73,1.73,1.6954,1.66945,1.6435,1.71,0.0146,0.04055,0.0665
18984,18984,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,...,1.0,21.02,21.02,21.4404,21.75570,22.0710,22.03,-0.5896,-0.27430,0.0410
18985,18985,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,...,1.0,21.02,21.02,21.4404,21.75570,22.0710,22.03,-0.5896,-0.27430,0.0410
18986,18986,"Ryan, Praharsh",22.03,25.0,1,200m,1998-10-18,None,Sprint,Male,...,1.0,21.02,21.02,21.4404,21.75570,22.0710,22.03,-0.5896,-0.27430,0.0410


In [1684]:
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan Jun Jie')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'SNG, MICHELLE', value='Sng, Michelle')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'SNG, SUAT LI, MICHELLE', value='Sng, Michelle')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'MUN, IVAN', value='Mun, Ivan')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LOW, JUN YU', value='Low, Jun Yu')

rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')
rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'LIM, OLIVER', value='Lim, Oliver')



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/180801585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/180801585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['NAME'] = rslt_df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t400

In [1685]:
rslt_df['PERF_SCALAR']=rslt_df['Delta5']/rslt_df['Metric']*100

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/1814660016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_df['PERF_SCALAR']=rslt_df['Delta5']/rslt_df['Metric']*100


In [1686]:
rslt_df.to_csv('checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)


In [1687]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


In [1688]:
spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [1689]:
spex_list

,V1,V2,V3,V4,Level
0,Goh Chui Ling,"Goh, Chui Ling",NaN,NaN,E3
1,Michelle Sng,"Sng, Michelle",NaN,NaN,E3
2,Quek Jun Jie Calvin,"Quek, Jun Jie Calvin",NaN,NaN,E3
3,"SOH RUI YONG, GUILLAUME","Soh Rui Yong, Guillaume",NaN,NaN,E3
4,Aaron Justin tan wen jie,"Tan Wen Jie, Aaron Justin",NaN,NaN,E3P
5,DANIEL LEOW SOON YEE,"Yee, Daniel Leow Soon",NaN,NaN,E3P
6,Joshua Chua,"Chua, Joshua",NaN,NaN,E3P
7,NG ZHI RONG RYAN RAPHAEL,"Ng Zhi Rong, Ryan Raphael","Raphael, Ryan",NaN,E3P
8,Wenli Rachel,"Wenli, Rachel",NaN,NaN,E3P
9,Wong YaoHan Melvin,"Wong YaoHan, Melvin",NaN,NaN,E3P


In [1690]:
spex1 = spex_list['V1'].dropna().tolist()
spex2 = spex_list['V2'].dropna().tolist()
spex3 = spex_list['V3'].dropna().tolist()
spex4 = spex_list['V4'].dropna().tolist()


In [1691]:
spex_athletes=spex1+spex2+spex3+spex4

In [1692]:
spex_athletes

['Goh Chui Ling',
 'Michelle Sng',
 'Quek Jun Jie Calvin',
 'SOH RUI YONG, GUILLAUME',
 'Aaron Justin tan wen jie',
 'DANIEL LEOW SOON YEE',
 'Joshua Chua',
 'NG ZHI RONG RYAN RAPHAEL',
 'Wenli Rachel',
 'Wong YaoHan Melvin',
 'XANDER HO ANN HENG',
 'Veronica Shanti Pereira',
 'Ang Chen Xiang',
 'KAMPTON KAM',
 'MARC BRIAN LOUIS',
 'Mark Lee Ren',
 'Reuben Rainer Lee Siong En',
 'ELIZABETH-ANN TAN SHEE RU',
 'Thiruben Thana Rajan',
 'Bhavna Gopikrishna',
 'Chloe Chee En-Ya',
 'Conrad Kangli Emery',
 'Harry Irfan Curran',
 'Huang weijun',
 'Jayden tan',
 'Koh Shun Yi Audrey',
 'LAAVINIA D/O JAIGANTH',
 'Lim Yee Chern Clara',
 'LOH DING RONG ANSON',
 'ONG YING TAT',
 'SONG EN XU REAGAN',
 'Subaraghav hari',
 'Teh Ying Shan',
 'YAN TEO',
 'ZHONG CHUHAN',
 'ESTHER TAY SHEE WEI',
 'Faith Ford',
 'Garrett Chua Je-An',
 'Lucas Fun',
 'Goh, Chui Ling',
 'Sng, Michelle',
 'Quek, Jun Jie Calvin',
 'Soh Rui Yong, Guillaume',
 'Tan Wen Jie, Aaron Justin',
 'Yee, Daniel Leow Soon',
 'Chua, Joshua',

In [1693]:
spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [1694]:
spex_athletes_casefold

['goh chui ling',
 'michelle sng',
 'quek jun jie calvin',
 'soh rui yong, guillaume',
 'aaron justin tan wen jie',
 'daniel leow soon yee',
 'joshua chua',
 'ng zhi rong ryan raphael',
 'wenli rachel',
 'wong yaohan melvin',
 'xander ho ann heng',
 'veronica shanti pereira',
 'ang chen xiang',
 'kampton kam',
 'marc brian louis',
 'mark lee ren',
 'reuben rainer lee siong en',
 'elizabeth-ann tan shee ru',
 'thiruben thana rajan',
 'bhavna gopikrishna',
 'chloe chee en-ya',
 'conrad kangli emery',
 'harry irfan curran',
 'huang weijun',
 'jayden tan',
 'koh shun yi audrey',
 'laavinia d/o jaiganth',
 'lim yee chern clara',
 'loh ding rong anson',
 'ong ying tat',
 'song en xu reagan',
 'subaraghav hari',
 'teh ying shan',
 'yan teo',
 'zhong chuhan',
 'esther tay shee wei',
 'faith ford',
 'garrett chua je-an',
 'lucas fun',
 'goh, chui ling',
 'sng, michelle',
 'quek, jun jie calvin',
 'soh rui yong, guillaume',
 'tan wen jie, aaron justin',
 'yee, daniel leow soon',
 'chua, joshua',

In [1695]:
top_performers=rslt_df.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [1696]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,BENCHMARK,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
14995,14995,"{80: 'Jirapat, KHANONTA\xa0', 79: 'Manatsada, ...",46.06,0.0,3,4 x 100m relay,None,None,Relay,Female,...,44.58,44.580,45.47160,46.140300,46.80900,46.06,-0.58840,0.080300,0.74900,1.680126
14978,14978,"{71: 'KANHAAUDOM, Methawi', 74: 'Joseph Obioma...",3:14.84,0.0,2,4 x 400m relay,None,None,Relay,Male,...,3:08.82,188.820,192.59640,195.428700,198.26100,194.84,-2.24360,0.588700,3.42100,1.811778
9555,9555,"{36: 'QUEK, CALVIN', 35: 'CHUA, JOSHUA', 37: '...",40.56,0.0,1,4 x 100m relay,None,None,Relay,Male,...,39.36,39.360,40.14720,40.737600,41.32800,40.56,-0.41280,0.177600,0.76800,1.951220
14968,14968,"{26: 'ROSLAN, ABDUL WAFIY', 29: 'ROSELI, RUSLE...",3:14.46,0.0,1,4 x 400m relay,None,None,Relay,Male,...,3:08.82,188.820,192.59640,195.428700,198.26100,194.46,-1.86360,0.968700,3.80100,2.013028
15001,15001,"{260: 'ZULKIFLI, ZAIDATUL HUSNIAH', 261: 'ALIA...",45.70,0.0,1,4 x 100m relay,None,None,Relay,Female,...,44.58,44.580,45.47160,46.140300,46.80900,45.70,-0.22840,0.440300,1.10900,2.487663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15077,15077,"Amjaram, Vinsensia Awutet",5.75m,26.0,1,Long jump,1998-06-04 00:00:00.000,None,Jump,Female,...,6.02,6.020,5.89960,5.809300,5.71900,5.75,-0.14960,-0.059300,0.03100,0.514950
14681,14681,"ANURA, ANDRE",14.93m,24.0,2,Triple jump,1999-06-12,None,Jump,Male,...,15.7,15.700,15.38600,15.150500,14.91500,14.93,-0.45600,-0.220500,0.01500,0.095541
15161,15161,"ANG, JAMIE EL-REDHA",10.80,20.0,5,100m,2004-01-06 00:00:00.000,None,Sprint,Male,...,10.443,10.443,10.65186,10.808505,10.96515,10.80,-0.14814,0.008505,0.16515,1.581442
14926,14926,"ALING, NUR AISHAH ROFINA",12.09,24.0,4,100m,1999-01-31,None,Sprint,Female,...,11.75,11.750,11.98500,12.161250,12.33750,12.09,-0.10500,0.071250,0.24750,2.106383


In [1707]:
top_performers.to_csv('top_checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)


In [1708]:
#spexed_list = top_performers.loc[~rslt_df['NAME'].str.casefold().isin(spex_athletes_casefold)]  # ~ means NOT IN. DROP spex carded athletes

In [1709]:
spexed_list=top_performers

In [1710]:
spexed_list.sort_values(['EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
spexed_list['overall_rank'] = 1
spexed_list['overall_rank'] = spexed_list.groupby(['EVENT', 'GENDER'])['overall_rank'].cumsum()

In [1711]:
#spexed_list=spexed_list[spexed_list['overall_rank']<15]

In [1712]:
spexed_list.to_csv('ranked_all_events.csv', sep=',', encoding='utf-8-sig', index=False)
